<a href="https://colab.research.google.com/github/2ovisa/AH2179/blob/main/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 1

*In this assignment, you will apply regression models to both problems and practice end-to-end model development, parameter tuning, and performance evaluation.*

##Task 1: Bus arrival delays

In [3]:
#Code copied from exercise 1
import pandas as pd

df = pd.read_csv('Exercise2BusData.csv')
df.head(10)

df = df.iloc[:1000]
df = df.drop(['Arrival_time', 'Stop_id', 'Bus_id', 'Line_id'], axis = 1)

#y = the variable we want to predict
#x = the variable that explains what y will be
x = df.drop(['Arrival_delay'], axis=1)
y = df['Arrival_delay']

*Split data into a training and test set*

In [4]:
#Code copied from exercise 1
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

##Create a linear regression model with XGBoost

In [5]:
#Code from: https://xgboost.readthedocs.io/en/stable/get_started.html
import xgboost as xgb
from xgboost import XGBRegressor
my_model = XGBRegressor()


my_model.fit(X_train, y_train)
y_pred = my_model.predict(X_test)

*Grid search to find out which parameters to use*


In [10]:
#Code from exercise and: https://www.kaggle.com/code/tilii7/hyperparameter-grid-search-with-xgboost/notebook
#https://xgboost.readthedocs.io/en/stable/parameter.html#parameters-for-linear-booster-booster-gblinear
from sklearn.model_selection import GridSearchCV

param_grid= {
    'n_estimators' :[800, 900, 1000, 1100],
    'learning_rate' : [0.001, 0.01, 0.02],
    'max_depth' : [2, 3, 4, 5]
}

grid_search = GridSearchCV(estimator = my_model, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_

print("Best parameters:", best_parameters)
print("Best score:", grid_search.best_score_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1000}
Best score: 0.9883039832115174


*Training the model with the best parameters from the grid search*

In [11]:
final_model = xgb.XGBRegressor(**best_parameters)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)

##Evaluation

In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Absolute Error: 10.36670207977295
Mean Squared Error: 197.3595733642578
R-squared: 0.9920626878738403


*Comparison to baseline*

RMSE from XGBoost is the root of MSE = 14.04847228.

The lower the RMSE & the MAE the better the model. The regression model with XGBoost has both a lower RMSE and MAE value then the baseline. This means that the predictions that the model makes are closer to the real values than what values from the baseline model are. The regression model with XGBoost does also have a higher R-squared value when you compare it to the baseline, this means that the XGBoost regression model is better at prediciting the mean of y.